In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, merge, Dense, Flatten
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, Sequential, model_from_json, load_model
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras import backend as K
import torch

%run data_preprocess.ipynb

C:\softwares\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#encode images 

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
#x = Flatten()(x)
encoded = Dense(128, activation='relu')(x)
# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

encoder = Model(input_img, encoded)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:


def prepare_data(X):
    prepared_data = []

    for i in range(X.shape[0]):
    
        input_im1 = process_image(X[i,0])
        input_im2 = process_image(X[i,1])
        
        encoded_img1 = encoder.predict(input_im1)
        encoded_img2 = encoder.predict(input_im2)

        
        encoded_img1 = encoded_img1.flatten()
        
        encoded_img2 = encoded_img2.flatten()
        
        mean1 = np.mean(encoded_img1 - encoded_img2)
        mean2 = np.mean(encoded_img1 + encoded_img2)
        euclidean_distance = np.sum((encoded_img1-encoded_img2)**2)
        im_sum = np.sum((encoded_img1-encoded_img2))

        data = [mean1,mean2,euclidean_distance,im_sum]
        result_im = list(((encoded_img1-encoded_img2)**2))
        result_im = result_im + data
        prepared_data.append(result_im)
        #prepared_data.append(data)
    
    
    return prepared_data


